In [1]:
from b2d.testbed.backend.jupyter import JupyterGui

In [2]:
import ipywidgets
import ipycanvas
import b2d
from plot_newtons_cradle import NewtonsCradle  as Example
gui_settings = {
    "id": Example
}
Example.run(JupyterGui, gui_settings=gui_settings)

Output()

In [ ]:
from plot_blender import Blender  as Example
gui_settings = {
    "id": Example
}
Example.run(JupyterGui, gui_settings=gui_settings)

In [ ]:
from plot_gauss_machine import GaussMachine  as Example
gui_settings = {
    "id": Example
}
Example.run(JupyterGui, gui_settings=gui_settings)

In [ ]:
from plot_color_mixing import ColorMixing  as Example
gui_settings = {
    "id": Example
}
Example.run(JupyterGui, gui_settings=gui_settings)

In [ ]:
from plot_function_shape import FunctionShape  as Example
res = (400,400)
gui_settings = {
    "id": Example
}
Example.run(JupyterGui, gui_settings=gui_settings)

In [3]:
from b2d.testbed import TestbedBase
import random
import numpy
import b2d
import math




class Keyboardx(TestbedBase):

    name = "the func shape"
    
    def __init__(self): 
        super(Keyboardx, self).__init__(gravity=(0,0))
    


        dimensions = [30,30]
        
        # the outer box
        box_shape = b2d.ChainShape()
        box_shape.create_loop([
                (0,0),
                (0,dimensions[1]),
                (dimensions[0],dimensions[1]),
                (dimensions[0],0)
            ]
        )
        box = self.world.create_static_body( position=(0, 0), shape = box_shape)

        self.rocket = self.world.create_dynamic_body(position=(dimensions[0]/2,10), 
            fixtures=b2d.fixture_def(
                shape=b2d.polygon_shape(box=[1,1]),
                density=1
            )
        )



        # particle system
        pdef = b2d.particle_system_def(viscous_strength=0.9,spring_strength=0.0, 
            damping_strength=100.5,pressure_strength=1.0,
            color_mixing_strength=0.05,density=0.1)

        psystem = self.world.create_particle_system(pdef)
        psystem.radius = 0.1
        psystem.damping = 0.5

        if False:
            emitter_def = b2d.RandomizedLinearEmitterDef()
            emitter_def.emite_rate = 200
            emitter_def.lifetime = 1
            emitter_def.size = (2,0.1)
            emitter_def.velocity = (0,-10)
            emitter_def.body = self.rocket
            emitter_def.transform = b2d.Transform(self.rocket.position, b2d.Rot(0))
            self.emitter = b2d.RandomizedLinearEmitter(psystem, emitter_def)
        else:
            angle_width = (math.pi * 2) / 16
            emitter_def = b2d.RandomizedRadialEmitterDef()
            emitter_def.emite_rate = 2000
            emitter_def.lifetime = 1.0
            emitter_def.enabled = False
            emitter_def.inner_radius = 1.0
            emitter_def.outer_radius = 1
            emitter_def.velocity_magnitude = 10.0
            emitter_def.start_angle =  math.pi/2 - angle_width /2.0
            emitter_def.stop_angle = math.pi/2 + angle_width /2.0
            emitter_def.body = self.rocket
            world_anchor =  self.rocket.get_world_point((0,-1.1))
            emitter_def.transform = b2d.Transform(world_anchor, b2d.Rot(math.pi))
            self.emitter = b2d.RandomizedRadialEmitter(psystem, emitter_def)



    def pre_step(self, dt):

        # get the world point of the emitters anchor point
        world_anchor = self.rocket.get_world_point((0,-1.1))


        self.emitter.position = world_anchor
        self.emitter.angle = self.rocket.angle  + math.pi
        # print(world_anchor, self.emitter.position)

        self.emitter.step(dt)

    def on_keyboard_down(self, keycode):
        print("key down", self.iter, keycode)
        if keycode[1] ==  'w':
            self.emitter.enabled = True
            return True
        return False


    def on_keyboard_up(self, keycode):
        print("key up", self.iter, keycode)
        if keycode[1] ==  'w':
            self.emitter.enabled = False
            return True
        return False

gui_settings = {
    "id": Keyboardx
}

ani = b2d.testbed.run(Keyboardx, gui_settings=gui_settings)
ani


Output()